In [1]:
import sys
import json
import numpy as np
params = {}
default = {'training_folder': '../../scenarios/dallas/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/dallas/', 'params': {}}


In [2]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [3]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [4]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(base_folder, 'model/')

In [5]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
    
cutoff = var['pathfinder']['cutoff']   

# inputs

In [6]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')
distribution = stepmodel.read_zippedpickles(model_folder + 'distribution')

sm.volumes = distribution.volumes

zone_before_clustering: 100%|██████████| 2/2 [00:00<00:00, 38.56it/s]


# od_set

In [7]:
v = sm.volumes.set_index(['origin', 'destination']).sum(axis=1)
od_set = set(v.loc[v>0].index)


# PT pathfinder

In [8]:
cutoff=7200
sm.step_pt_pathfinder(
    broken_routes=True,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    cutoff=cutoff,
    verbose=False,
)
ppf = sm.publicpathfinder

start publicpathfinder


route_type {'bus'}: 100%|██████████| 15/15 [00:05<00:00,  2.75it/s]                            

path_analysis


In [9]:

sm.pt_los.set_index(['origin', 'destination'])['pathfinder_session'].value_counts()


route_breaker    5369
best_path        4306
mode_breaker     2072
Name: pathfinder_session, dtype: int64

In [10]:
sm.step_road_pathfinder(method='aon', access_time='time',time_column='time',od_set=od_set,path_analysis=True)

In [11]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

path_analysis: 100%|██████████| 11747/11747 [00:00<00:00, 54495.34it/s]


In [12]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

car_los: 100%|██████████| 30/30 [00:10<00:00,  2.94it/s]        


In [13]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined